In [54]:
%run functions.ipynb

In [35]:
test_path = 'tss/timestamp_interval=1562220000'
base_path = 'tss'
july4 = 'tss/timestamp_interval=1562220000'
june3 = 'tss/timestamp_interval=1559590000'
sep3 = 'tss/timestamp_interval=1567480000'
aug3 = 'tss/timestamp_interval=1564850000'
all_files = ['tss/' + p for p in os.listdir('tss/') if p.startswith('timestamp')]
all_files2 = ['part/' + p for p in os.listdir('part/') if p.startswith('timestamp')]

In [5]:
def query_dispatcher(query):
    if query.startswith('TSel'):
        return temporal_selection(query)
    if query.startswith('WSel'):
        return window_selection(query)
    if query.startswith('Shift'):
        return shift(query)
    if query.startswith('TAgg'):
        return temporal_aggregation(query)
    if query.startswith('WAgg'):
        return window_aggregation(query)

## Temporal Selection
Return: a time series where we replace the original value with an empty value (!) if the predicate is not satisfied

In [23]:
query = 'TSel[activity==Rue]'

In [25]:
def temporal_selection(query):
    predicate = query[5:-1]
    if '==' in predicate:
        s = predicate.split('==')
        return s[0], s[1], 'streq'

In [26]:
attribute, value, q = query_dispatcher(query)

In [39]:
%run unpack_ts.ipynb

+--------------------+--------------------+--------------------+---------------+------------------+-----------------+------------------+
|           tablet_id|            activity|         coordinates|timestamp_first|global_granularity|inner_granularity|timestamp_interval|
+--------------------+--------------------+--------------------+---------------+------------------+-----------------+------------------+
|[CH184703920, CH1...|[Rue, Métro, Rue,...|[[48.8919, 2.2318...|     1566878400|             month|           minute|        1564850000|
+--------------------+--------------------+--------------------+---------------+------------------+-----------------+------------------+

root
 |-- tablet_id: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- activity: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- coordinates: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull 

In [40]:
if q == 'streq':
    df1.select('timestamp', attribute).withColumn(attribute, when(col(attribute)==value, value).otherwise('!')).show(5)

+----------+--------+
| timestamp|activity|
+----------+--------+
|1566878400|     Rue|
|1566878460|       !|
|1566878520|     Rue|
|1566878580|       !|
|1566878640|     Rue|
+----------+--------+
only showing top 5 rows



## Window Selection
Load from disk only timeseries within specific time interval

In [4]:
query = 'WSel[01/06/2019,30/07/2019]'
def window_selection(query):
    predicate = query[5:-1]
    s = predicate.split(',')
    return s[0], s[1], None

In [5]:
start, end, _ = query_dispatcher(query)

In [6]:
import datetime

date_format = '%d/%m/%Y'
start_date = datetime.datetime.strptime(start, date_format)
end_date = datetime.datetime.strptime(end, date_format)
start_epoch = int(start_date.timestamp())
end_epoch = int(end_date.timestamp())

In [7]:
import os
def test_time_interval(file, start, end):
    if '=' not in file:
        return False
    f = file.split("=")
    timestamp = int(f[1])
    if timestamp >= start and timestamp <= end:
        return True
    else:
        return False

files = [f for f in os.listdir('tss/') if test_time_interval(f, start_epoch, end_epoch)]

In [8]:
df = unpack('tss', files)

In [89]:
df.show()

+--------+-----------+--------------------+----------+
|activity|  tablet_id|         coordinates| timestamp|
+--------+-----------+--------------------+----------+
|Domicile|CH184703916|[48.8365266666667...|1560307680|
|     Rue|CH184703916|[48.8359416666667...|1560307740|
|     Bus|CH184703916| [48.83648, 2.52973]|1560307800|
|     Rue|CH184703916|[48.8363583333333...|1560307860|
|  Bureau|CH184703916|[48.8362033333333...|1560307920|
|     Rue|CH184703916|[48.8293983333333...|1560307980|
|   Métro|CH184703916|[48.8316016666667...|1560308040|
|Domicile|CH184703916|[48.826745, 2.355...|1560308100|
|     Rue|CH184703916|[48.8270466666667...|1560308160|
| Magasin|CH184703916|[48.8269083333333...|1560308220|
|     Rue|CH184703916|[48.8270083333333...|1560308280|
|     Bus|CH184703916|[48.82687, 2.3549...|1560308340|
|Domicile|CH184703916|[48.8269683333333...|1560308400|
|     Rue|CH184703916|[48.8270716666667...|1560308460|
|   Train|CH184703916|[48.8270883333333...|1560308520|
|     Rue|

## Temporal Projection
Apply a linear function on each value of the time series

In [66]:
query = 'TProj[PM10*2]'

In [67]:
def temporal_projection(query):
    predicate = query[6:-1]
    for op in ops.keys():
        s = predicate.split(op)
        if len(s) == 2:
            if '.' in s[1]:
                n = float(s[1])
            else:
                n = int(s[1])
            return s[0], op, n

In [70]:
attribute, op, number = temporal_projection(query)

In [75]:
df = unpack('part', all_files2)
df.show()

+-----+----+-----+----+----+--------+--------------------+--------------------+----------+
|PM2_5|PM10|PM1_0| NO2|  BC|activity|               event|         coordinates| timestamp|
+-----+----+-----+----+----+--------+--------------------+--------------------+----------+
|   12|  13|   13|NULL|1509|Domicile|Ouverture De Fenêtre|[48.83652667, 2.5...|1560314880|
|    4|   4|    4|NULL|1105|     Rue|                NULL|[48.83594167, 2.5...|1560314940|
|    1|   1|    1|NULL| 917|     Bus|                NULL| [48.83648, 2.52973]|1560315000|
|    1|   1|    1|NULL| 974|     Rue|                NULL|[48.83635833, 2.5...|1560315060|
|    2|   2|    2|NULL| 846|  Bureau|Ouverture De Fenêtre|[48.83620333, 2.5...|1560315120|
|    0|   0|    0|NULL| 114|     Rue|                NULL|[48.82939833, 2.3...|1560315180|
|   18|  20|   20|NULL|2198|   Métro|                NULL|[48.83160167, 2.3...|1560315240|
|    0|   0|    0|NULL| 364|Domicile|                NULL|[48.826745, 2.355...|1560315300|

In [73]:
df = df.withColumn(attribute, ops.get(op)(col(attribute), number))

In [74]:
df.show()

+-----+-----+-----+----+----+--------+--------------------+--------------------+----------+
|PM2_5| PM10|PM1_0| NO2|  BC|activity|               event|         coordinates| timestamp|
+-----+-----+-----+----+----+--------+--------------------+--------------------+----------+
|   12| 26.0|   13|NULL|1509|Domicile|Ouverture De Fenêtre|[48.83652667, 2.5...|1560314880|
|    4|  8.0|    4|NULL|1105|     Rue|                NULL|[48.83594167, 2.5...|1560314940|
|    1|  2.0|    1|NULL| 917|     Bus|                NULL| [48.83648, 2.52973]|1560315000|
|    1|  2.0|    1|NULL| 974|     Rue|                NULL|[48.83635833, 2.5...|1560315060|
|    2|  4.0|    2|NULL| 846|  Bureau|Ouverture De Fenêtre|[48.83620333, 2.5...|1560315120|
|    0|  0.0|    0|NULL| 114|     Rue|                NULL|[48.82939833, 2.3...|1560315180|
|   18| 40.0|   20|NULL|2198|   Métro|                NULL|[48.83160167, 2.3...|1560315240|
|    0|  0.0|    0|NULL| 364|Domicile|                NULL|[48.826745, 2.355...|

## Shift
Applies a shift to each timestamp of the timeseries

In [126]:
query = 'Shift[1 hour]'

In [127]:
def shift(query):
    predicate = query[6:-1]
    s = predicate.split(' ')
    return int(s[0]), s[1], None

In [128]:
n, duration, _ = shift(query)

In [129]:
step = calculate_step_in_seconds(duration) * n
step

3600

In [122]:
df = unpack(base_path, [all_files])
df.show()

+--------+-----------+--------------------+----------+
|activity|  tablet_id|         coordinates| timestamp|
+--------+-----------+--------------------+----------+
|Domicile|CH184703916|[48.8365266666667...|1560307680|
|     Rue|CH184703916|[48.8359416666667...|1560307740|
|     Bus|CH184703916| [48.83648, 2.52973]|1560307800|
|     Rue|CH184703916|[48.8363583333333...|1560307860|
|  Bureau|CH184703916|[48.8362033333333...|1560307920|
|     Rue|CH184703916|[48.8293983333333...|1560307980|
|   Métro|CH184703916|[48.8316016666667...|1560308040|
|Domicile|CH184703916|[48.826745, 2.355...|1560308100|
|     Rue|CH184703916|[48.8270466666667...|1560308160|
| Magasin|CH184703916|[48.8269083333333...|1560308220|
|     Rue|CH184703916|[48.8270083333333...|1560308280|
|     Bus|CH184703916|[48.82687, 2.3549...|1560308340|
|Domicile|CH184703916|[48.8269683333333...|1560308400|
|     Rue|CH184703916|[48.8270716666667...|1560308460|
|   Train|CH184703916|[48.8270883333333...|1560308520|
|     Rue|

In [130]:
df1 = df.withColumn('timestamp_shifted', df.timestamp + step).withColumnRenamed('timestamp', 'timestamp_original')
df1.show()

+--------+-----------+--------------------+------------------+-----------------+
|activity|  tablet_id|         coordinates|timestamp_original|timestamp_shifted|
+--------+-----------+--------------------+------------------+-----------------+
|Domicile|CH184703916|[48.8365266666667...|        1560307680|       1560311280|
|     Rue|CH184703916|[48.8359416666667...|        1560307740|       1560311340|
|     Bus|CH184703916| [48.83648, 2.52973]|        1560307800|       1560311400|
|     Rue|CH184703916|[48.8363583333333...|        1560307860|       1560311460|
|  Bureau|CH184703916|[48.8362033333333...|        1560307920|       1560311520|
|     Rue|CH184703916|[48.8293983333333...|        1560307980|       1560311580|
|   Métro|CH184703916|[48.8316016666667...|        1560308040|       1560311640|
|Domicile|CH184703916|[48.826745, 2.355...|        1560308100|       1560311700|
|     Rue|CH184703916|[48.8270466666667...|        1560308160|       1560311760|
| Magasin|CH184703916|[48.82

In [131]:
df_joined = df1.join(df, df1.timestamp_shifted == df.timestamp, "inner").drop('timestamp')
df_joined.show()

+--------+-----------+--------------------+------------------+-----------------+----------+-----------+--------------------+
|activity|  tablet_id|         coordinates|timestamp_original|timestamp_shifted|  activity|  tablet_id|         coordinates|
+--------+-----------+--------------------+------------------+-----------------+----------+-----------+--------------------+
|Domicile|CH184703916|[48.8365266666667...|        1560307680|       1560311280|  Domicile|CH184703920|[48.8478393555, 2...|
|     Rue|CH184703916|[48.8359416666667...|        1560307740|       1560311340|       Rue|CH184703942|[48.883237, 2.324...|
|     Bus|CH184703916| [48.83648, 2.52973]|        1560307800|       1560311400|       Rue|CH184703920|[48.8147087097, 2...|
|     Rue|CH184703916|[48.8363583333333...|        1560307860|       1560311460|    Bureau|CH184703920|[48.8147087097, 2...|
|  Bureau|CH184703916|[48.8362033333333...|        1560307920|       1560311520|       Rue|CH184703949|[48.8811035156, 2...|


## Temporal Intersection, Difference & Union

## Temporal Aggregation

In [38]:
query = 'TAgg[1 day,avg(PM10)]'

In [39]:
def temporal_aggregation(query):
    predicate = query[5:-1]
    s = predicate.split(',')
    s2 = s[1].split('(')
    return s[0], s2[1][:-1], s2[0]

In [40]:
duration, attribute, agg = query_dispatcher(query)

In [36]:
df = unpack('part', all_files2)

In [37]:
df.show()

+-----+----+-----+----+----+--------+--------------------+--------------------+----------+
|PM2_5|PM10|PM1_0| NO2|  BC|activity|               event|         coordinates| timestamp|
+-----+----+-----+----+----+--------+--------------------+--------------------+----------+
|   12|  13|   13|NULL|1509|Domicile|Ouverture De Fenêtre|[48.83652667, 2.5...|1560314880|
|    4|   4|    4|NULL|1105|     Rue|                NULL|[48.83594167, 2.5...|1560314940|
|    1|   1|    1|NULL| 917|     Bus|                NULL| [48.83648, 2.52973]|1560315000|
|    1|   1|    1|NULL| 974|     Rue|                NULL|[48.83635833, 2.5...|1560315060|
|    2|   2|    2|NULL| 846|  Bureau|Ouverture De Fenêtre|[48.83620333, 2.5...|1560315120|
|    0|   0|    0|NULL| 114|     Rue|                NULL|[48.82939833, 2.3...|1560315180|
|   18|  20|   20|NULL|2198|   Métro|                NULL|[48.83160167, 2.3...|1560315240|
|    0|   0|    0|NULL| 364|Domicile|                NULL|[48.826745, 2.355...|1560315300|

In [41]:
step = calculate_step_from_string(duration)
step

86400

In [42]:
first_timestamp = df.select('timestamp').first()[0]

In [43]:
df1 = df.withColumn('first_timestamp', lit(first_timestamp))

In [44]:
df1 = df1.withColumn('group', ((df1.timestamp - df1.first_timestamp)/lit(step)).cast("integer")).drop(
    'first_timestamp')

In [45]:
if agg == 'count':
    group = df1.groupBy('group', attribute).count()
elif agg == 'avg':
    group = df1.groupBy('group').agg(mean(attribute))   

In [46]:
group.sort('group').show()

+-----+------------------+
|group|         avg(PM10)|
+-----+------------------+
|    0| 40.61538461538461|
|    1|3.3333333333333335|
|    3| 7.833333333333333|
|    5|2.3333333333333335|
+-----+------------------+



## Window Aggregation

In [88]:
query = 'WAgg[1 day,avg(PM10)]'

In [89]:
def window_aggregation(query):
    predicate = query[5:-1]
    s = predicate.split(',')
    s1 = s[0].split(' ')
    step = calculate_step_in_seconds(s1[1]) * int(s1[0])
    s2 = s[1].split('(')
    return step, s2[1][:-1], s2[0]

In [90]:
step, attribute, agg = query_dispatcher(query)

In [91]:
window = Window.orderBy('timestamp').rangeBetween(0, step)

In [84]:
df = unpack('part', all_files2)

In [92]:
all_original_columns = df.columns

In [93]:
column_name = agg + '(' + attribute + ')'
if agg == 'count':
    df1 = df.withColumn(column_name, (collect_list(attribute).over(window)))
    df1 = df1.select('*', explode(column_name).alias('exploded')).filter(col(attribute) == col('exploded'))\
    .groupBy(*all_original_columns, column_name).count()
    df1 = df1.drop(column_name, 'exploded')
elif agg == 'avg':
    df1 = df.withColumn(column_name, avg(attribute).over(window))

In [94]:
df1.show(10, False)

+-----+----+-----+----+----+--------+--------------------+--------------------------+----------+-----+
|PM2_5|PM10|PM1_0|NO2 |BC  |activity|event               |coordinates               |timestamp |count|
+-----+----+-----+----+----+--------+--------------------+--------------------------+----------+-----+
|12   |13  |13   |NULL|1509|Domicile|Ouverture De Fenêtre|[48.83652667, 2.529348333]|1560314880|1    |
|4    |4   |4    |NULL|1105|Rue     |NULL                |[48.83594167, 2.530051667]|1560314940|1    |
|1    |1   |1    |NULL|917 |Bus     |NULL                |[48.83648, 2.52973]       |1560315000|3    |
|1    |1   |1    |NULL|974 |Rue     |NULL                |[48.83635833, 2.529766667]|1560315060|2    |
|2    |2   |2    |NULL|846 |Bureau  |Ouverture De Fenêtre|[48.83620333, 2.52943]    |1560315120|1    |
|0    |0   |0    |NULL|114 |Rue     |NULL                |[48.82939833, 2.315678333]|1560315180|2    |
|18   |20  |20   |NULL|2198|Métro   |NULL                |[48.83160167, 2